# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import sys

### Importing Data as pandas dataframe and preprocessing it

In [84]:
df = pd.read_csv(r"C:\Users\Aman-Verma\Downloads\ASP Transition Matrix and HMM\DWJ\2010-20.csv",error_bad_lines=False)

In [85]:
df = df.drop('High',axis=1)
df = df.drop('Low',axis=1)
df = df.drop('Adj Close',axis=1)
df = df.drop('Volume',axis=1)

In [86]:
df.describe()

,Open,Close
count,2516.000000,2516.000000
mean,17601.997740,17606.741572
std,5148.425941,5147.050113
min,9686.480469,9686.480469
25%,12977.189942,12981.207276
50%,17011.075196,17008.229492
75%,21426.617188,21411.106934
max,28675.339844,28645.259766


In [87]:
df.head()

,Date,Open,Close
0,2010-01-04,10430.690430,10583.959961
1,2010-01-05,10584.559570,10572.019531
2,2010-01-06,10564.719727,10573.679688
3,2010-01-07,10571.110352,10606.860352
4,2010-01-08,10606.400391,10618.190430


### Setting up the parameters for the model 

In [88]:
df['Difference'] = df['Close'] - df['Open']
df['Difference Mod'] = df['Difference'].abs()

In [89]:
df.head()

,Date,Open,Close,Difference,Difference Mod
0,2010-01-04,10430.690430,10583.959961,153.269531,153.269531
1,2010-01-05,10584.559570,10572.019531,-12.540039,12.540039
2,2010-01-06,10564.719727,10573.679688,8.959961,8.959961
3,2010-01-07,10571.110352,10606.860352,35.750000,35.750000
4,2010-01-08,10606.400391,10618.190430,11.790039,11.790039


In [90]:
df.describe()

,Open,Close,Difference,Difference Mod
count,2516.000000,2516.000000,2516.000000,2516.000000
mean,17601.997740,17606.741572,4.743833,96.232042
std,5148.425941,5147.050113,141.217945,103.444347
min,9686.480469,9686.480469,-1041.839843,0.000000
25%,12977.189942,12981.207276,-51.772461,26.560058
50%,17011.075196,17008.229492,9.199707,64.465332
75%,21426.617188,21411.106934,72.541992,129.612549
max,28675.339844,28645.259766,1020.718750,1041.839843


### Setting up the range  

In [91]:
criteria = [df['Difference'].between(-800, -48.1), df['Difference'].between(-48.1, 48.1), df['Difference'].between(48.1,1000)]
values = ['Down', 'Stagnant', 'Up']

df['Movement'] = np.select(criteria, values, 'Stagnant')

In [92]:
df.head()

,Date,Open,Close,Difference,Difference Mod,Movement
0,2010-01-04,10430.690430,10583.959961,153.269531,153.269531,Up
1,2010-01-05,10584.559570,10572.019531,-12.540039,12.540039,Stagnant
2,2010-01-06,10564.719727,10573.679688,8.959961,8.959961,Stagnant
3,2010-01-07,10571.110352,10606.860352,35.750000,35.750000,Stagnant
4,2010-01-08,10606.400391,10618.190430,11.790039,11.790039,Stagnant


### Showing number of States  

In [93]:
df['Movement'].value_counts()

Stagnant    1014
Up           849
Down         653
Name: Movement, dtype: int64

In [94]:
z = list(df['Movement'])

### Transition Probability Matrix 

In [95]:
pd.crosstab(pd.Series(z[1:],name='Tomorrow'),
            pd.Series(z[:-1],name='Today'),normalize=0)

Today,Down,Stagnant,Up
Tomorrow,,,
Down,0.281776,0.384380,0.333844
Stagnant,0.206114,0.437870,0.356016
Up,0.306604,0.376179,0.317217


In [96]:
from itertools import islice

def window(seq, n=2):
    "Sliding window width n from seq.  From old itertools recipes."""
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [97]:
import pandas as pd

pairs = pd.DataFrame(window(z), columns=['Today', 'Tomorrow'])
counts = pairs.groupby('Today')['Tomorrow'].value_counts()
probs = (counts).unstack()

### Transition Count Matrix

In [98]:
probs

Tomorrow,Down,Stagnant,Up
Today,,,
Down,184,209,260
Stagnant,251,444,319
Up,218,361,269


# End